In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import tree
from collections import defaultdict
import math
import re

# Load data

In [ ]:
source_train_df = pd.read_csv("data/train.csv")
source_test_df = pd.read_csv("data/test.csv")

# Derived value extraction

In [ ]:
# Set some data as categorical if it indeed is
source_train_df["Pclass"] = source_train_df["Pclass"].astype('category')
source_test_df["Pclass"] = source_test_df["Pclass"].astype('category')

In [ ]:
def add_name_derived_features(df):
    df["FamilyName"] = df["Name"].map(lambda name: name.split(",")[0])
    df["NameHasQuotes"] = df["Name"].map(lambda name: 1 if name.find("\"") != -1 else 0)
    df["NameHasParentheses"] = df["Name"].map(lambda name: 1 if name.find("(") != -1 else 0)

def add_title(df):
    
    def _get_title(name):
        title_and_name = name.split(",")[1]
        return title_and_name.split(".")[0].strip()
    
    df["Title"] = df["Name"].map(lambda name : _get_title(name) )
    
    title_to_title_family = {
         "Capt":       "officer",
         "Col":        "officer",
         "Major":      "officer",
         "Dr":         "officer",
         "Rev":        "officer",
         "Jonkheer":   "snob",
         "Don":        "snob",
         "Sir" :       "snob",
         "the Countess":"snob",
         "Dona":       "snob",
         "Lady" :      "snob",
         "Mme":        "married",
         "Ms":         "married",
         "Mrs" :       "married",
         "Miss" :      "single",
         "Mlle":       "single",
         "Mr" :        "man",
         "Master" :    "boy"
    }
    
    df["TitleGroup"] = df["Title"].map(lambda title: title_to_title_family[title])
    # Fill age nan for each group
    median_age_by_pclass_and_title = defaultdict(dict)
    for pclass in [1, 2, 3]:
        for title in title_to_title_family.keys():
            median_age_by_pclass_and_title[pclass][title] = df[(df.Pclass == pclass) & (df.Title == title)]["Age"].mean()
    
    df["Age"] = [
        median_age_by_pclass_and_title[row["Pclass"]][row["Title"]] if math.isnan(row.Age) else row["Age"] 
        for _, row in df.iterrows()
    ]
    
    df["isChild"] = [1. if row.Age < 15 else 0. for _, row in df.iterrows()]
    df["isOld"] = [1. if row.Age >= 65 else 0. for _, row in df.iterrows()]


# Add cabin deck
def add_deck(df):
    
    df['Cabin'] = df['Cabin'].map(lambda x : "Unknown" if x == "" or x is np.nan else x)
    df['Deck_Unknown'] = df['Cabin'].map(lambda x : 1 if x == "Unknown" else 0)
    
    deck_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G']
    for deck in deck_list:
        df['Deck_{}'.format(deck)] = df['Cabin'].map(lambda x : 1 if deck in x else 0)


def add_room_number(df):
    
    def _room_number(cabin):
        result = re.compile(r"([0-9]+)").search(cabin)
        if result:
            return result.group()
        return 0
    
    df['Room'] = df['Cabin'].map(lambda cell: _room_number(cell) if cell != "Unknown" else np.nan).astype(float) 
    
    def _has_room(room):
        if room is np.nan or math.isnan(room):
            return 0
        return 1
    
    df['HasRoom'] = df['Room'].map(_has_room)
    room_mean = df['Room'].mean()
    df['Room'] = df['Room'].fillna(room_mean)
    df['Room'] = df['Room'].astype(int)
    df['RoomInFront'] = df['Room'].map(lambda cell: 1 if cell <= room_mean else 0)
    df['RoomInBack'] = df['Room'].map(lambda cell: 1 if cell > room_mean else 0)
            

def add_family_size(df):
    df['FamilySize'] = df['SibSp'] + df['Parch']
    df['Singleton'] = df['FamilySize'].map(lambda s: 1 if s == 1 else 0)
    df['SmallFamily'] = df['FamilySize'].map(lambda s: 1 if 2<=s<=4 else 0)
    df['LargeFamily'] = df['FamilySize'].map(lambda s: 1 if 5<=s else 0)


def add_fare_per_person(df):
    df['FarePerPerson'] = df['Fare']/(df['FamilySize']+1)

    
def add_ticket_derived_features(df):
    ticket_count = df["Ticket"].value_counts()
    df["TicketCount"] = df["Ticket"].map(lambda ticket : ticket_count[ticket])
    df["TicketPrefix"] = df["Ticket"].map(lambda ticket : ticket.split(" ")[0] if " " in ticket else "")
    df["TicketNumber"] = df["Ticket"].map(lambda ticket : ticket.split(" ")[1] if " " in ticket else ticket)
    df["TicketNumberSize"] = df["TicketNumber"].map(lambda ticket : len(ticket))
    
    def _get_first_digit(ticket_number):
        if re.match(r"\d+", ticket_number):
            return "{}".format(ticket_number)[0]
        return np.nan
    
    df["TicketFirstDigit"] = df["TicketNumber"].map(_get_first_digit).astype(float)


def add_new_features(df):
    add_name_derived_features(df)
    add_title(df)
    add_deck(df)
    add_family_size(df)
    add_fare_per_person(df)
    add_room_number(df)
    add_ticket_derived_features(df)

In [ ]:
add_new_features(source_train_df)
add_new_features(source_test_df)

ticket_prefix_categories = list(source_train_df["TicketPrefix"].unique())

In [ ]:
print(source_train_df.columns)

In [ ]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
source_train_df.head(20)

In [ ]:
# Fare
# It is known that fare == 0 is an error, so will be replacing it to nan
source_train_df["Fare"] = source_train_df["Fare"].map(lambda _fare: np.nan if _fare == 0.0 else _fare)
source_test_df["Fare"] = source_test_df["Fare"].map(lambda _fare: np.nan if _fare == 0.0 else _fare)

In [ ]:
# Replace nan values
def replace_nan_values(df):
    return df.fillna(df.median())
    
train_no_nans_df = replace_nan_values(source_train_df)
test_no_nans_df = replace_nan_values(source_test_df)

In [ ]:
# Drop columns we are not interested
columns_to_drop = ["Name", "Ticket", "Cabin"]

simplified_train_df = train_no_nans_df.drop(columns_to_drop, axis=1)
simplified_test_df = test_no_nans_df.drop(columns_to_drop, axis=1)

In [ ]:
# Nominal attributes are changed to values
categorical_columns = ["Pclass", "Sex", "Embarked", "Title", "TitleGroup", "TicketPrefix"]

expanded_train_df = pd.get_dummies(simplified_train_df, columns=categorical_columns)
expanded_columns = expanded_train_df.columns.values.tolist()

expanded_test_df = pd.get_dummies(simplified_test_df, columns=categorical_columns)

In [ ]:
# In case any column of test is not present in train, set it to zero
all_columns = set(expanded_train_df.columns).union(set(expanded_test_df.columns))
for column in all_columns:
    if column not in expanded_train_df.columns:
        expanded_train_df[column] = 0
    if column not in expanded_test_df.columns:
        expanded_test_df[column] = 0

In [ ]:
print(expanded_train_df.columns)
print(expanded_test_df.columns)

In [ ]:
assert len(expanded_train_df.columns) == len(expanded_test_df.columns)

In [ ]:
expanded_train_df.head()

# Check if output directory exists

In [ ]:
import os
if not os.path.exists("results"):
    os.makedirs("results")

# View correlations

In [ ]:
# Features that could have a role in the classification
possible_selected_features = [
    "PassengerId",
    "Age",
    "Pclass_1", "Pclass_2", "Pclass_3",
    "Sex_female", "Sex_male",
    "NameHasQuotes", "NameHasParentheses",
    "Fare",
    "FarePerPerson",
    "SibSp",
    "Parch",
    "TicketCount",
    "TicketFirstDigit",
    "FamilySize",
    "Singleton",
    "SmallFamily",
    "LargeFamily",
    "HasRoom",
    "Room",
    "RoomInBack",
    "RoomInFront",
    "isChild",
    "isOld",
    "Deck_A", "Deck_B", "Deck_C", "Deck_D", "Deck_E", "Deck_F", "Deck_T", "Deck_G"
] +\
["TitleGroup_{}".format(family_group) for family_group in ["officer", "snob", "married", "single", "man", "boy"]] +\
["TicketPrefix_{}".format(ticket_prefix) for ticket_prefix in ticket_prefix_categories]


In [ ]:
expanded_train_df[["Survived"]+possible_selected_features].corr()["Survived"].sort_values(ascending=False)

# Prepare raw data for algorithms

In [ ]:
# Features that will have a role in the classification
selected_features = [
    "Age",
    "Pclass_1", "Pclass_2", "Pclass_3",
    "Sex_female", "Sex_male",
    "NameHasQuotes", "NameHasParentheses",
    "Fare",
    "FarePerPerson",
    "SibSp",
    "Parch",
    "TicketCount",
    "TicketFirstDigit",
    "FamilySize",
    "Singleton",
    "SmallFamily",
    "LargeFamily",
    "HasRoom",
    "RoomInBack",
    "RoomInFront",
    "isChild",
    "isOld",
    "Deck_A", "Deck_B", "Deck_C", "Deck_D", "Deck_E", "Deck_F", "Deck_T", "Deck_G"
] +\
["TitleGroup_{}".format(family_group) for family_group in ["officer", "snob", "married", "single", "man", "boy"]] +\
["TicketPrefix_{}".format(ticket_prefix) for ticket_prefix in ticket_prefix_categories]


In [ ]:
# X and Y are the input and output of the classifier algorithm
train_y = train_no_nans_df.Survived.astype(int).values

# test_x and train_x must have the same number of columns and
# test_x has no "Survived" column so we must drop it from train_x
train_x = expanded_train_df[selected_features].values

test_x = expanded_test_df[selected_features].values

# Scaler type

In [ ]:
scaler = StandardScaler

# Base classification functions

In [ ]:
class ClassifierRunner(object):
    
    def __init__(self, pipeline, parameters, debug=False):
        self.pipeline = pipeline
        self.parameters = parameters
        self.grid_search = GridSearchCV(self.pipeline, self.parameters, cv=5)
        self.debug = debug
        
    def fit(self, X, y):
        self.grid_search.fit(X, y)

    @property
    def best_params(self):
        return self.grid_search.best_params_
        
    def get_scores(self, X, y, num_folds=5):
        scores = cross_val_score(self.grid_search.best_estimator_, X, y, cv=num_folds)
        return scores
    
    @property
    def feature_importances(self):
        classifier_step_index = 0
        for step_name, step_process in self.grid_search.best_estimator_.steps:
            if step_name == "classifier":
                break
            classifier_step_index += 1
        feature_importances = self.grid_search.best_estimator_.steps[classifier_step_index][1].feature_importances_
        return sorted(zip(feature_importances, selected_features), reverse=True)
    
    def predict(self, X_test):
        prediction = self.grid_search.predict(X_test)
        return prediction

    def apply_predicition_to_df(self, X_test, test_df, output_filename):
        prediction = self.predict(X_test)
        # Add the prediction to the test dataset
        test_df_classified = test_df.assign(Survived=list(prediction))
        # Save 
        test_df_classified.to_csv(output_filename, columns=["PassengerId", "Survived"], index=False)
        

# Centroid

In [ ]:
# Basic decision tree Pipeline
pipeline = Pipeline([
        ('minmaxscaler', scaler()),
        ('classifier', NearestCentroid())
    ])

parameters = {
    'classifier__metric': ["manhattan", "euclidean"], 
    'classifier__shrink_threshold': [None, .05, .1, .2, .5, .55, .6, .7, .74, .75, .77, .8, .85, .9, 1],
}

classifier = ClassifierRunner(pipeline=pipeline, parameters=parameters)

classifier.fit(train_x, train_y)

print ("Best parameters found: ")
print (classifier.best_params)

scores = classifier.get_scores(train_x, train_y)
print ("Expected performance: {:.2f}% (+/-{:.2f}).".format(np.mean(scores)*100., np.std(scores)*100.))

classifier.apply_predicition_to_df(test_x, source_test_df, output_filename="results/test_classified_with_nearest_centroid.csv")


# Basic decision tree

In [ ]:
# Basic decision tree Pipeline
pipeline = Pipeline([
        ('classifier', tree.DecisionTreeClassifier())
    ])

max_depths = [10, 20, 30, 40, 50, 70, 100, 150, 200, 300, 400, 1000]
parameters = {
    'classifier__max_depth': max_depths, 
    'classifier__criterion': ["gini", "entropy"],
    'classifier__splitter': ["best", "random"],
    'classifier__min_samples_split':[2, 3, 4, 5, 7, 10, 15, 20, 25],
    'classifier__random_state': [1],
    'classifier__max_features': [2, 5, 10, 20, 40, "auto", "sqrt", "log2", None]
}

classifier = ClassifierRunner(pipeline=pipeline, parameters=parameters)

classifier.fit(train_x, train_y)

print ("Best parameters found: ")
print (classifier.best_params)

feature_importances = classifier.feature_importances
print("Selected features by importance: {}".format(feature_importances))

scores = classifier.get_scores(train_x, train_y)
print ("Expected performance: {:.2f}% (+/-{:.2f}).".format(np.mean(scores)*100., np.std(scores)*100.))

classifier.apply_predicition_to_df(test_x, source_test_df, output_filename="results/test_classified_with_dt.csv")

# MLPClassifier

In [ ]:
# Basic NN classifier Pipeline
pipeline = Pipeline([
        ('minmaxscaler', scaler()),
        ('classifier', MLPClassifier())
    ])

hidden_layer_sizes = [100, 100, 100]
parameters = {
    'classifier__hidden_layer_sizes': hidden_layer_sizes, 
    'classifier__activation': ["identity", "logistic", "tanh", "relu"],
    'classifier__solver': ["lbfgs", "sgd", "adam"],
    'classifier__learning_rate' : ["constant", "invscaling", "adaptive"]
}

classifier = ClassifierRunner(pipeline=pipeline, parameters=parameters)

classifier.fit(train_x, train_y)

print ("Best parameters found: ")
print (classifier.best_params)

scores = classifier.get_scores(train_x, train_y)
print ("Expected performance: {:.2f}% (+/-{:.2f}).".format(np.mean(scores)*100., np.std(scores)*100.))

classifier.apply_predicition_to_df(test_x, source_test_df, output_filename="results/test_classified_with_nn.csv")

# Gaussian Naive Bayes

In [ ]:
# Basic NB classifier Pipeline
pipeline = Pipeline([
    ('classifier', GaussianNB())
])

parameters = {
    'classifier__priors': [None]
}

classifier = ClassifierRunner(pipeline=pipeline, parameters=parameters)

classifier.fit(train_x, train_y)

print ("Best parameters found: ")
print (classifier.best_params)

scores = classifier.get_scores(train_x, train_y)
print ("Expected performance: {:.2f}% (+/-{:.2f}).".format(np.mean(scores)*100., np.std(scores)*100.))

classifier.apply_predicition_to_df(test_x, source_test_df, output_filename="results/test_classified_with_nb.csv")

# Predict test values with KNN

In [ ]:
# KNN Pipeline
pipeline = Pipeline([
        ('minmaxscaler', scaler()),
        ('classifier', KNeighborsClassifier())
    ])

k_values = [1, 2, 3, 4, 5, 7, 10, 15, 20, 25, 30, 40, 50]
parameters = {
    'classifier__n_neighbors': k_values, 
    'classifier__weights': ["uniform", "distance"],
    'classifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

classifier = ClassifierRunner(pipeline=pipeline, parameters=parameters)

classifier.fit(train_x, train_y)

print ("Best parameters found: ")
print (classifier.best_params)

scores = classifier.get_scores(train_x, train_y)
print ("Expected performance: {:.2f}% (+/-{:.2f}).".format(np.mean(scores)*100., np.std(scores)*100.))

classifier.apply_predicition_to_df(test_x, source_test_df, output_filename="results/test_classified_with_knn.csv")

# Predict test values with RandomForest

In [ ]:
# Random forest Pipeline

pipeline = Pipeline([
        ('classifier', RandomForestClassifier())
    ])


parameters = {
                 'classifier__max_depth' : [4, 6, 12, 20, None],
                 'classifier__criterion': ['gini', 'entropy'],
                 'classifier__n_estimators': [2, 10, 50, 100],
                 'classifier__max_features': ['sqrt', 'auto', 'log2', None],
                 'classifier__min_samples_split': [2, 3, 10],
                 'classifier__min_samples_leaf': [1, 3, 10],
                 'classifier__bootstrap': [True, False],
                 'classifier__n_jobs': [-1]
                 }

classifier = ClassifierRunner(pipeline=pipeline, parameters=parameters)

classifier.fit(train_x, train_y)

print ("Best parameters found: ")
print (classifier.best_params)

feature_importances = classifier.feature_importances
print("Selected features by importance: {}".format(feature_importances))

scores = classifier.get_scores(train_x, train_y)
print ("Expected performance: {:.2f}% (+/-{:.2f}).".format(np.mean(scores)*100., np.std(scores)*100.))

classifier.apply_predicition_to_df(test_x, source_test_df, output_filename="results/test_classified_with_rf.csv")


# Predict with SVM

In [ ]:
# Random forest Pipeline
from sklearn.pipeline import Pipeline
from sklearn import svm


tuned_parameters = [
    {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]},
    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}
]

scores = ['precision', 'recall']

for score in scores:

    clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(train_x, train_y)

    print "Best parameters for SVM"
    print clf.best_params_
    
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    
    print('Best parameters: {}'.format(clf.best_params_))

    NFOLDS = 5
    scores = cross_val_score(clf.best_estimator_, train_x, train_y, cv=NFOLDS)
    print ("Expected performance: {:.2f}% (+/-{:.2f}).".format(np.mean(scores)*100., np.std(scores)*100.))

    # Make the prediction over the test set
    svm_prediction = clf.predict(test_x)

    # Add the prediction to the test dataset
    test_classified_with_svm = source_test_df.assign(Survived=list(svm_prediction))

    # Save to upload to Kaggle
    test_classified_with_svm.to_csv("results/test_classified_with_svm_{}.csv".format(score), columns=["PassengerId", "Survived"], index=False)
